Team: Pandas

Group members: Francesca-Zhoufan Li, Elena Sorina Lupu, Nikhil Ranganathan

# Install and Import Packages

In [ ]:
!pip install iqplot
!pip install surprise
!pip install nevergrad

In [ ]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

from math import pi
import numpy as np
import pandas as pd

np.random.seed(42)

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import seaborn as sns

sns.set_theme(style="white", context="talk")

import iqplot
import bokeh.io
from bokeh.plotting import figure
from bokeh.layouts import column, gridplot
from bokeh.models import (
    ColorBar,
    ColorMapper,
    LinearColorMapper,
    Ticker,
    ColumnDataSource, 
    Label, 
    LabelSet
)

bokeh.io.output_notebook()

import holoviews as hv
from holoviews import dim
from holoviews import opts

# import bebi103

hv.extension("bokeh")

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

import surprise
from surprise import SVD, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate

import nevergrad as ng


# Load and Clean Up Data

In [ ]:
def load_train_test(trainortest):
    """Load train or test data"""
    return pd.read_csv(
        "https://raw.githubusercontent.com/lakigigar/Caltech-CS155-2021/main/projects/project2/data/"
        + trainortest,
        sep="\t",
        header=None,
        names=["USER", "MOVIE", "RATING"],
    )

In [ ]:
def load_data(f_data, f_train, f_test, f_movies):
    """Load the user and movie data, FZL modified"""

    data = load_train_test(f_data)
    train = load_train_test(f_train)
    test = load_train_test(f_test)

    movies = pd.read_csv(
        "https://raw.githubusercontent.com/lakigigar/Caltech-CS155-2021/main/projects/project2/data/"
        + f_movies,
        encoding="latin-1",
        sep="\t",
        header=None,
        names=[
            "MOVIE_ID",
            "TITLE",
            "UNKNOWN",
            "ACTION",
            "ADVENTURE",
            "ANIMATION",
            "CHILDREN",
            "COMEDY",
            "CRIME",
            "DOCUMENTARY",
            "DRAMA",
            "FANTASY",
            "FILM-NOIR",
            "HORROR",
            "MUSICAL",
            "MYSTERY",
            "ROMANCE",
            "SCI-FI",
            "THRILLER",
            "WAR",
            "WESTERN",
        ],
    )

    movies.loc[movies.TITLE == "unknown", "TITLE"] = "MOVIE_ID: " + movies.loc[
        movies.TITLE == "unknown", "MOVIE_ID"
    ].astype("str")

    return data, train, test, movies

In [ ]:
def check_dup(df, df_details):
    """Check fi there are duplicated entries for each dataframe"""
    print(f"There are {sum((df.duplicated())*1)} duplicate entires in {df_details}")

In [ ]:
data, Y_train_df, Y_test_df, movies = load_data(
    "data.txt", "train.txt", "test.txt", "movies.txt"
)

check_dup(Y_train_df, "Y_train_df")
check_dup(Y_test_df, "Y_test_df")

# Basic Visualization

## All MovieLens Dataset

In [ ]:
sum_df = pd.DataFrame(movies.set_index(["MOVIE_ID", "TITLE"]).sum(axis=0)).reset_index()
sum_df.columns = ["Genres", "Counts"]
sum_df = sum_df.sort_values(["Counts"], ascending=False).reset_index(drop=True)

plt.figure(figsize=(15, 8))
x = np.array(list(sum_df.Genres))
y1 = sum_df.Counts.values
sum_bar = sns.barplot(x=x, y=y1, palette="crest_r")

for index, row in sum_df.iterrows():
    sum_bar.text(index, row.Counts, row.Counts, color="black", ha="center")

sum_bar.set(xlabel="Genres", ylabel="Counts", title="Summary of gernre counts")
sum_bar.set_xticklabels(sum_bar.get_xticklabels(), rotation=90)
plt.show()

In [ ]:
def plot_genre_heat(df, title_details):
    """Plot heatmap based on if belonging to the genre"""
    plt.figure(figsize=(10, len(df)*0.02))
    movie_heat = sns.heatmap(
        df,
        yticklabels=False,
        cmap=[(1, 1, 1), (0.14573579, 0.29354139, 0.49847009)],
        cbar_kws=dict(
            use_gridspec=False,
            shrink=0.2,
            ticks=[0, 1],
            label="if the movie belongs to the genre",
            # location="top"
        ),
    )

    movie_heat.set(
        xlabel="Genres", ylabel="Movies", title="Summary of "+ title_details
    )
    plt.show()
    return movie_heat

In [ ]:
all_movie_heat = plot_genre_heat(movies.iloc[:, 1:].set_index("TITLE"),
                                 "genres for each movie")

In [ ]:
def get_types(movies):
    """Get how many types of different genre combinations"""
    genre_comb = movies.iloc[:,2:].drop_duplicates()
    cols_name = list(genre_comb.columns)
    return genre_comb.sort_values(by=cols_name)[cols_name]

In [ ]:
movie_type_heat = plot_genre_heat(get_types(movies),
                                 "genres combinations")

## All ratings in the MovieLens Dataset

In [ ]:
data_all = iqplot.histogram(
    data=data, q="RATING", bins="exact", title="Rating for all movies"
)
bokeh.io.show(data_all)

In [ ]:
def plot_heat_rate(
    df, title_details, x_tick_scale=None, y_tick_scale=None, ifreorder=False
):
    """Plot heatmap where x-axis is the users, y-axis is the movie id,
    and the color corresponds to the rating"""

    try:
        if "TITLE" in df.columns:
            nrow = len(df.TITLE.unique())
            df_heat = df.pivot("TITLE", "USER", "RATING")
        elif "MOVIE_ID" in df.columns:
            nrow = len(df.MOVIE_ID.unique())
            df_heat = df.pivot("MOVIE_ID", "USER", "RATING")
        elif "MOVIE" in df.columns:
            nrow = len(df.MOVIE.unique())
            df_heat = df.pivot("MOVIE", "USER", "RATING")
        else:
            nrow = -1
    except:
        print("Resolving duplicating issue")
        df_heat = (
            df[["TITLE", "USER", "RATING"]]
            .drop_duplicates()
            .reset_index(drop=True)
            .pivot_table(values="RATING", index=["TITLE", "USER"], aggfunc="mean")
            .unstack(1)
        )
        df_heat.columns = [user_id for r, user_id in df_heat.columns]

    if nrow == None:
        nscale = -20
    elif nrow < 4:
        nscale = 4
    elif nrow > 1000:
        nscale = 0.01
        y_tick_scale = 50
    else:
        nscale = 0.6
    p_height = nrow * nscale

    if len(df.USER.unique()) > 50:
        x_tick_scale = 50
    
    if ifreorder:
        df_heat = df_heat.reindex(df.TITLE.unique())

    plt.figure(figsize=(20, p_height))
    rating_heat = sns.heatmap(
        df_heat,
        cmap="crest",
        cbar_kws=dict(
            use_gridspec=False,
            shrink=0.2,
            ticks=list(range(1, 6)),
            label="rating",
            location="top",
        ),
    )

    if x_tick_scale != None:
        rating_heat.xaxis.set_major_locator(ticker.MultipleLocator(x_tick_scale))
        rating_heat.xaxis.set_major_formatter(ticker.ScalarFormatter())

    if y_tick_scale != None:
        rating_heat.yaxis.set_major_locator(ticker.MultipleLocator(y_tick_scale))
        rating_heat.yaxis.set_major_formatter(ticker.ScalarFormatter())
    
    rating_heat.set(xlabel="USER", title=f"Summary of {title_details}")
    plt.show()

    return rating_heat

In [ ]:
def plot_rating_cat(df, cat):
    """Plot the rating grouped by user or movie"""
    p = iqplot.stripbox(
        data=df.sort_values(cat),
        q="RATING",
        cats=cat,
        plot_width=len(df[cat].unique())*10,
        palette=list(
            sns.color_palette("crest_r", len(df[cat].unique())).as_hex()
        ),
        jitter=True,
        marker_kwargs=dict(alpha=0.05),
        q_axis="y",
        title="Rating per " + cat,
    )
    p.xaxis.major_label_orientation = pi/2
    bokeh.io.show(p)

In [ ]:
def plot_ecdfs(df, q, ifshow=True):
    """Plot the ecdf based on popularity and average rating"""
    counts_ecdf = iqplot.ecdf(
            data=get_pop_movie(data, movies),
            q="RATING_COUNTS",
            title="ECDF for all movie popularity",
        )
    
    rate_ecdf = iqplot.ecdf(
            data=get_top_rate(data, movies), q="RATING", title="ECDF for all rating"
        )

    if ifshow:
        bokeh.io.show(counts_ecdf)
        bokeh.io.show(rate_ecdf)

    return counts_ecdf, rate_ecdf

In [ ]:
rating_heat_all = plot_heat_rate(data,
                                 "full user movie rating",
                                 x_tick_scale=50, y_tick_scale=50)

In [ ]:
plot_rating_cat(data, "USER")

In [ ]:
plot_rating_cat(data, "MOVIE")

In [ ]:
counts_ecdf, rate_ecdf = plot_ecdfs(data, movies)

## All ratings of the ten most popular (rated) movies

In [ ]:
def get_pop_movie(data, movies):
    """Get the number and title of rated movies"""
    movie_count = pd.DataFrame(data.MOVIE.value_counts()).reset_index()
    movie_count.columns = ["MOVIE_ID", "RATING_COUNTS"]
    return movie_count

In [ ]:
def merge_title(df, movies, ifgenres=False):
    """Add movie titles"""
    if "MOVIE" in df.columns:
        df = df.rename(columns={"MOVIE": "MOVIE_ID"})
    if not ifgenres:
        movie_comb = movies.iloc[:, :2]
    else:
        movie_comb = movies

    df = df.merge(movie_comb, left_on="MOVIE_ID", right_on="MOVIE_ID", how="left")

    df.loc[df.TITLE.isnull(), "TITLE"] = (
        "MOVIE_ID: " + df.loc[df.TITLE.isnull(), "MOVIE_ID"]
    )

    return df

In [ ]:
def get_top_pop_movie_data(data, movies, topn):
    """Get the top rated movie ratings"""
    data_lists = []
    movie_count = get_pop_movie(data, movies)
    for i in movie_count.MOVIE_ID[:topn]:
        data_lists.append(data[data.MOVIE==i])
    return merge_title(pd.concat(data_lists), movies)

In [ ]:
def get_topn_str(df, topn):
    """Convert topn to all if None or number"""
    if topn == None:
        topn = len(df)
        return "all"
    else:
        return str(topn)

In [ ]:
def hv_render(plot):
    """Render holoviews plots"""
    # Take out the Bokeh object
    p = hv.render(plot)
    # Display using Bokeh
    bokeh.io.show(p)

In [ ]:
def plot_vio(df, title_details, topn):
    """Plot violin plots"""
    topn_str = get_topn_str(df, topn)
    ncols = len(df.TITLE.unique())
    if ncols < 700:
        col_w = 50
    else:
        col_w = 20
    
    hv_violin = hv.Violin(
        (df["TITLE"], df["RATING"]), ["TITLE"], "Value",
    ).opts(
        violin_color=hv.dim("TITLE").str(),
        cmap="GnBu_r",
        ylabel="RATING",
        title=f"Rating violin plot for the {topn_str} {title_details} movies",
        ylim=(-1, 7),
        show_legend=False,
        colorbar=True,
        # cut=1,
        # bandwidth=1,
        # inner='stick',
        box_alpha=0.5,
        xrotation=90,
        width=col_w * len(df["TITLE"].unique()),
        height=300 + df.TITLE.map(len).max()*5,
    )

    hv_render(hv_violin)
    
    return hv_violin

In [ ]:
def plot_topn_pop_rating(df, title_details, topn):
    """Plot the top n most popular/rated movies"""

    topn_str = get_topn_str(df, topn)
    
    ncols = len(df.TITLE.unique())
    if ncols < 100:
        col_w = 50
    else:
        col_w = 20
    p_w = ncols * col_w

    if p_w < 500:
        p_w = 500

    topn_pop = iqplot.stripbox(
        data=df,
        q="RATING",
        cats="TITLE",
        palette=list(sns.color_palette("crest_r", topn).as_hex()),
        jitter=True,
        top_level="box",
        q_axis="y",
        plot_width=p_w,
        marker_kwargs=dict(alpha=0.05),
        title=f"Rating stripbox plot for the {topn_str} {title_details} movies",
    )

    mapper = LinearColorMapper(
        palette=list(sns.color_palette("crest", topn).as_hex()), low=1, high=topn,
    )
    color_bar = ColorBar(
        color_mapper=mapper,
        padding=0,
        location=(0, 0),
        title="Most rated",
        title_standoff=10,
    )

    topn_pop.add_layout(color_bar, "right")
    topn_pop.xaxis.major_label_orientation = pi / 2
    topn_pop.xaxis.axis_label = "MOVIE"
    bokeh.io.show(topn_pop)
    return topn_pop

In [ ]:
def plot_hist_all(df, title_details, topn, ifoverlay=True):
    """Plot summary histogram"""
    if ifoverlay:
        set_cat = "TITLE"
        set_kind = "step"
        set_palette = list(
            sns.color_palette("crest_r", len(df.TITLE.unique())).as_hex()
        )
    else:
        set_cat = None
        set_kind = "step_filled"
        set_palette = [sns.color_palette("crest", 5).as_hex()[-1]] * len(df)

    topn_str = get_topn_str(df, topn)
    h = iqplot.histogram(
        data=df,
        bins="exact",
        q="RATING",
        cats=set_cat,
        kind=set_kind,
        palette=set_palette,
        title=f"Rating histogram for the {topn_str} {title_details} movies",
    )

    # comment out for colab
    # h.add_layout(h.legend[0], "right") 

    bokeh.io.show(h)
    return h

In [ ]:
def plot_hist_list(df):
    """Plot a list of hist depaned on title"""

    titles = df.TITLE.unique()
    hists = [None] * len(titles)
    for c, t in enumerate(titles):
        hists[c] = iqplot.histogram(
            data=df[df.TITLE == t],
            bins="exact",
            q="RATING",
            title=t,
            plot_width=200,
            plot_height=150,
        )

    grid = gridplot(hists, ncols=10)

    bokeh.io.show(column(grid))
    return hists

In [ ]:
def plot_plots(
    df,
    topn,
    title_details,
    x_tick_scale=None,
    y_tick_scale=None,
    ifvio=True,
    ifstripbox=True,
    ifheat=True,
    iftotallhist=True,
    ifoverlayhist=True,
    ifindhist=True,
    ifreorder=True,
):
    """Generate the stripbox or box, heatmap, and histograms"""

    topn_str = get_topn_str(df, topn)
    
    if ifvio:
        v = plot_vio(df, title_details, topn)
            
    if ifstripbox:
        try:
            poporrate = plot_topn_pop_rating(df, title_details, topn)
        except:
            poporrate = plot_top_rate(df, title_details, topn)
    
    if ifheat:
        heat = plot_heat_rate(
            df,
            f"{title_details} {topn_str} movies",
            x_tick_scale=x_tick_scale,
            y_tick_scale=y_tick_scale,
            ifreorder=ifreorder,
        )
    
    if iftotallhist:
        h_all = plot_hist_all(df,  title_details, topn, ifoverlay=False)
        
    if ifoverlayhist:
        h_overlay = plot_hist_all(df,  title_details, topn, ifoverlay=True)

    if ifindhist:
        hists = plot_hist_list(df)
   

In [ ]:
topn = 10
topn_pop_df = get_top_pop_movie_data(data, movies, topn)
plot_plots(topn_pop_df, topn, "top rated", x_tick_scale=50)

## All ratings of the ten best movies with the highest average ratings

In [ ]:
def get_top_rate(data, movies):
    """Sort by top avg rating"""
    return (
        data.set_index(["MOVIE"])
        .groupby(["MOVIE"])
        .mean()
        .sort_values(["RATING"], ascending=False)
        .reset_index()
    )

In [ ]:
def get_top_rate_full(data, movies, topn):
    """Get the top rated movies"""
    top_rate = get_top_rate(data, movies)

    df_list = []
    
    for i in top_rate.MOVIE[:topn]:
        df_list.append(data[data.MOVIE==i])
    
    return merge_title(pd.concat(df_list), movies)

In [ ]:
def plot_top_rate(df, title_details, topn):
    """Plot strip plots of the top rated movies"""
    
    if topn == None:
        topn = len(df)
        topn_str = "all"
    else:
        topn_str = topn
        
    ncols = len(df.TITLE.unique())
    if ncols < 20:
        col_w = 50
    else:
        col_w = 10
    p_w = ncols*col_w
    
    if p_w < 200:
        p_w = 200
        
    topn_rate = iqplot.strip(
        data=df,
        q="RATING",
        cats="TITLE",
        palette=list(sns.color_palette("crest_r", 
                                       len(df.RATING.unique())).as_hex()),
        jitter=True,
        q_axis="y",
        plot_width=p_w,
        marker_kwargs=dict(alpha=0.5),
        color_column="RATING",
        title=f"Rating for the {title_details} {topn_str} movies",
    )

    topn_rate.xaxis.major_label_orientation = pi / 2
    topn_rate.xaxis.axis_label = "MOVIE"
    bokeh.io.show(topn_rate)
    return topn_rate

In [ ]:
topn = 10
topn_rate_df = get_top_rate_full(data, movies, topn)
plot_plots(topn_rate_df, topn, "top average rating" ,ifvio=False)

## All ratings of movies from different genres of choice

In [ ]:
topn = None
genre_df = merge_title(data, movies, ifgenres=True)
genres = movies.columns[2:]
for g in genres:
    g_df = genre_df[genre_df[g] == 1]
    plot_plots(g_df, topn, g, ifstripbox=False, ifoverlayhist=False, 
               ifindhist=False, ifreorder=False)

# Matrix Factorization

## Prep

### Matrix factorization and data prep

In [ ]:
def grad_U(Ui, Yij, Vj, reg, eta):
    """
    Takes as input Ui (the ith row of U), a training point Yij, the column
    vector Vj (jth column of V^T), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Ui multiplied by eta.
    """
    return (1-reg*eta)*Ui + eta * Vj * (Yij - np.dot(Ui,Vj))     

def grad_V(Vj, Yij, Ui, reg, eta):
    """
    Takes as input the column vector Vj (jth column of V^T), a training point Yij,
    Ui (the ith row of U), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Vj multiplied by eta.
    """
    return (1-reg*eta)*Vj + eta * Ui * (Yij - np.dot(Ui,Vj))

def get_err(U, V, Y, reg=0.0):
    """
    Takes as input a matrix Y of triples (i, j, Y_ij) where i is the index of a user,
    j is the index of a movie, and Y_ij is user i's rating of movie j and
    user/movie matrices U and V.

    Returns the mean regularized squared-error of predictions made by
    estimating Y_{ij} as the dot product of the ith row of U and the jth column of V^T.
    """
    # Compute mean squared error on each data point in Y; include
    # regularization penalty in error calculations.
    # We first compute the total squared squared error
    err = 0.0
    for (i,j,Yij) in Y:
        err += 0.5 *(Yij - np.dot(U[i-1], V[:,j-1]))**2
    # Add error penalty due to regularization if regularization
    # parameter is nonzero
    if reg != 0:
        U_frobenius_norm = np.linalg.norm(U, ord='fro')
        V_frobenius_norm = np.linalg.norm(V, ord='fro')
        err += 0.5 * reg * (U_frobenius_norm ** 2)
        err += 0.5 * reg * (V_frobenius_norm ** 2)
    # Return the mean of the regularized error
    return err / float(len(Y))

def train_model(M, N, K, eta, reg, Y, eps=0.0001, max_epochs=300):
    """
    Given a training data matrix Y containing rows (i, j, Y_ij)
    where Y_ij is user i's rating on movie j, learns an
    M x K matrix U and N x K matrix V such that rating Y_ij is approximated
    by (UV)_ij.

    Uses a learning rate of <eta> and regularization of <reg>. Stops after
    <max_epochs> epochs, or once the magnitude of the decrease in regularized
    MSE between epochs is smaller than a fraction <eps> of the decrease in
    MSE after the first epoch.

    Returns a tuple (U, V, err) consisting of U, V, and the unregularized MSE
    of the model.
    """
    
    np.random.seed(42)
    
    # Initialize U, V  
    U = np.random.random((M,K)) - 0.5
    V = np.random.random((K,N)) - 0.5
    size = Y.shape[0]
    delta = None
    indices = np.arange(size)    
    for epoch in range(max_epochs):
        # Run an epoch of SGD
        before_E_in = get_err(U, V, Y, reg)
        np.random.shuffle(indices)
        for ind in indices:
            (i,j, Yij) = Y[ind]
            # Update U[i], V[j]
            U[i-1] = grad_U(U[i-1], Yij, V[:,j-1], reg, eta)
            V[:,j-1] = grad_V(V[:,j-1], Yij, U[i-1], reg, eta);
        # At end of epoch, print E_in
        E_in = get_err(U, V, Y, reg)
        print("Epoch %s, E_in (regularized MSE): %s"%(epoch + 1, E_in))

        # Compute change in E_in for first epoch
        if epoch == 0:
            delta = before_E_in - E_in

        # If E_in doesn't decrease by some fraction <eps>
        # of the initial decrease in E_in, stop early            
        elif before_E_in - E_in < eps * delta:
            break
    return (U, V, get_err(U, V, Y))

In [ ]:
def get_MNY(Y_train_df, Y_test_df):
    """Return Y_train, Y_test, M, N, 
    where M is unique user IDs and N is unique movie IDs"""
    return (
        Y_train_df.to_numpy(),
        Y_test_df.to_numpy(),
        int(max(max(Y_train_df.USER), max(Y_test_df.USER))),
        int(max(max(Y_train_df.MOVIE), max(Y_test_df.MOVIE))),
    )

In [ ]:
def get_UV_proj(U, V):
    """Get the UV projection"""   
    A, sigma, B = np.linalg.svd(V,  full_matrices=False)
    A_two_cols = A[:, 0:2]

    U_proj = A_two_cols.transpose()@U.transpose()
    V_proj = A_two_cols.transpose()@V
    
    return U_proj, V_proj

In [ ]:
def get_top_pop_movie_data(data, movies, topn):
    """Get the top rated movie ratings"""
    data_lists = []
    movie_count = get_pop_movie(data, movies)
    for i in movie_count.MOVIE_ID[:topn]:
        data_lists.append(data[data.MOVIE == i])
    return merge_title(pd.concat(data_lists), movies)


def get_topn_pop_n_rate_proj(
    df, rankcols, topn=10, cutoff=5, mostorleast="most", genres=None
):
    """Return top n rated and or pop movies, with and without cut off"""

    df = df[df.RATING_COUNTS >= cutoff]

    if genres != None:
        df = df[df[genres] == 1]

    # if ascending
    if mostorleast == "most":
        TorF = False
    else:
        TorF = True

    if isinstance(rankcols, list) and len(rankcols) > 1:
        ifasc = [TorF] * len(rankcols)
    elif (isinstance(rankcols, list) and len(rankcols) == 1) or isinstance(
        rankcols, str
    ):
        ifasc = TorF

    return df.sort_values(by=rankcols, ascending=ifasc).iloc[:topn, :]

In [ ]:
def comb_rate_pop(data, movies, ifgenres=True):
    """Combine rating and popularity and add titles"""
    return merge_title(
        (
            get_top_rate(data, movies)
            .rename(columns={"RATING": "AVG_RATING"})
            .merge(get_pop_movie(data, movies), left_on="MOVIE", right_on="MOVIE_ID")[
                ["MOVIE_ID", "AVG_RATING", "RATING_COUNTS"]
            ]
            .sort_values(by="MOVIE_ID")
        ),
        movies,
        ifgenres=ifgenres,
    )

def comb_proj_rate_pop(proj, data, movies, ifgenres=True):
    """Combine the projections with all other info"""
    proj_df = pd.DataFrame(proj.T, columns=["proj_1", "proj_2"])
    comb = proj_df.merge(
        comb_rate_pop(data, movies, ifgenres=ifgenres),
        left_index=True,
        right_index=True,
    )
    comb.RATING_COUNTS = comb.RATING_COUNTS.astype(np.float)
    return comb

In [ ]:
Y_train, Y_test, M, N = get_MNY(Y_train_df, Y_test_df)

### Surprise prep

In [ ]:
def use_surprise(
    Y_train_df, n_factors, n_epochs, lr_all, reg_all, ifbiased, ifreg_bubi
):
    """Use surprise to find pu, qi"""

    reader = Reader(rating_scale=(1, 5))
    train_dataset = Dataset.load_from_df(Y_train_df, reader)

    train_dataset_object = train_dataset.build_full_trainset()

    if ifreg_bubi:
        reg_bu = reg_all
        reg_bi = reg_all
    else:
        reg_bu = None
        reg_bi = None
    
    if ifbiased:
        reg_all = 0
    
    algo = SVD(
        n_factors=n_factors,
        n_epochs=n_epochs,
        lr_all=lr_all,
        reg_all=reg_all,
        reg_bu=reg_bu,
        reg_bi=reg_bi,
        biased=ifbiased,
        random_state=42,
    )
    cross_validate(algo, train_dataset, measures=["RMSE"], cv=5, verbose=True)

    algo.fit(train_dataset_object)

    print("pu shape", algo.pu.shape)
    print("qi shape", algo.qi.shape)

    pred = []
    for rows in Y_test_df.iterrows():
        uid = str(rows[1][0])
        iid = str(rows[1][1])
        rui = rows[1][2]

        # get a prediction for specific users and items.
        pred.append(algo.predict(uid, iid, r_ui=rui))

    return surprise.accuracy.rmse(pred), algo.pu[:,:2].T, algo.qi[:,:2].T

### Plotting prep

In [ ]:
def ecdf_vals(df, cat):
    """Compute x and y values for plotting an ECDF"""
    data = df[cat]
    return pd.DataFrame(
        {"x": np.sort(data), "ECDF": np.arange(1, len(data) + 1) / len(data)}
    )

def plot_pop_hist(meta_rate_pop):
    """Histogram for total rating counts, aka popularity"""
    count_hist = hv.render(
        (
            hv.Histogram(
                np.histogram(
                    meta_rate_pop["RATING_COUNTS"], int(len(meta_rate_pop) / 10),
                )
            )
            .opts(width=800, height=90,)
            .relabel()
            .opts(xaxis="top", yticks=[0, 300], line_color="grey", fill_color="grey",)
        )
    )
    count_hist.xaxis.major_label_text_font_size = "0pt"
    count_hist.xaxis.axis_label = None
    return count_hist

def plot_pop_ecdf(meta_rate_pop, title_details):
    """ecdf for total rating counts, aka popularity"""
    counts_ecdf = iqplot.ecdf(
        data=meta_rate_pop,
        q="RATING_COUNTS",
        plot_width=800,
        plot_height=160,
        x_axis_location="above",
        title="Movie-user projection " + title_details,
        palette=["#808080"],
        marker_kwargs=dict(alpha=0.5),
    )
    counts_ecdf.yaxis[0].ticker = [0, 0.5, 1]
    counts_ecdf.xgrid.visible = False
    counts_ecdf.ygrid.visible = False
    counts_ecdf.x_range.range_padding = 0.038
    counts_ecdf.title.text_font_size = "24px"
    counts_ecdf.title.align = "center"
    return counts_ecdf

def plot_avg_hist(meta_rate_pop):
    """Histogram for average rating"""
    avg_hist, avg_hist_edges = np.histogram(
        meta_rate_pop["AVG_RATING"], int(len(meta_rate_pop) / 10),
    )
    avg_hist_p = figure(plot_width=115, plot_height=800, y_axis_location="right")
    avg_hist_p.quad(
        top=avg_hist_edges[1:],
        bottom=avg_hist_edges[:-1],
        left=0,
        right=avg_hist,
        line_alpha=0,
        fill_color=list(sns.color_palette("GnBu_r", len(avg_hist)).as_hex()),
    )
    avg_hist_p.yaxis.axis_label = None
    avg_hist_p.xaxis.axis_label = "Frequency"
    avg_hist_p.y_range.flipped = True
    avg_hist_p.yaxis[0].ticker = list(np.linspace(1, 5, 9))
    avg_hist_p.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks
    avg_hist_p.yaxis.major_label_text_font_size = "0pt"
    avg_hist_p.xaxis[0].ticker = [0, 50, 100]
    avg_hist_p.xgrid.visible = False
    avg_hist_p.ygrid.visible = False
    avg_hist_p.y_range.range_padding = 0.025
    return avg_hist_p

def plot_rate_ecdf(data, movies):
    """Plot ecdf for all rating"""

    rate_ecdf = figure(plot_width=150, plot_height=800, y_axis_location="right")
    rate_ecdf_val = ecdf_vals(get_top_rate(data, movies), "RATING")
    rate_ecdf_source = ColumnDataSource(rate_ecdf_val)

    exp_cmap = LinearColorMapper(
        palette=list(sns.color_palette("GnBu", len(rate_ecdf_val["ECDF"])).as_hex()),
        low=0,
        high=1,
    )
    rate_ecdf.circle(
        "ECDF",
        "x",
        source=rate_ecdf_source,
        line_color=None,
        fill_color={"field": "ECDF", "transform": exp_cmap},
    )

    rate_ecdf.yaxis.axis_label = "AVE_RATING"
    rate_ecdf.xaxis.axis_label = "ECDF"
    rate_ecdf.xaxis[0].ticker = [0, 0.5, 1]
    rate_ecdf.yaxis[0].ticker = list(np.linspace(1, 5, 9))
    rate_ecdf.xgrid.visible = False
    rate_ecdf.ygrid.visible = False
    rate_ecdf.y_range.range_padding = 0.025
    
    return rate_ecdf

def get_hists_ecdfs(meta_rate_pop, title_details):
    """Plot the summary histogram and ecdf for 
    based on popularity and the average rating"""

    # histogram for total rating counts, aka popularity
    count_hist = plot_pop_hist(meta_rate_pop)

    # ecdf for total rating counts, aka popularity
    counts_ecdf = plot_pop_ecdf(meta_rate_pop, title_details)

    # histogram for average rating
    avg_hist_p = plot_avg_hist(meta_rate_pop)
    
    # plot ecdf for all rating
    rate_ecdf = plot_rate_ecdf(data, movies)
    
    return count_hist, counts_ecdf, avg_hist_p, rate_ecdf

In [ ]:
def plot_V_pts(comb_V_df):
    """plot V - movie as scatter plots"""
    popts = opts.Points(
        color="AVG_RATING",
        size=dim("RATING_COUNTS") * 0.2,
        alpha=0.5,
        line_alpha=1,
        line_width=2,
        width=800,
        height=800,
        cmap="GnBu",
        colorbar=True,
    )

    return hv.Points(
        comb_V_df,
        kdims=["proj_1", "proj_2"],
        vdims=["AVG_RATING", "RATING_COUNTS"],
        label="movie projection",
    ).opts(popts)

def plot_U_pts(comb_U_df):
    """plot U - users as scatter plots"""

    return hv.Points(
        comb_U_df,
        kdims=["proj_1", "proj_2"],
        vdims=["AVG_RATING", "RATING_COUNTS"],
        label="user projection",
    ).opts(
        fill_color="grey",
        fill_alpha=0.25,
        line_color="grey",
        line_width=0.1,
        show_legend=True,
    )

def plot_selected_V(
    df, legend_label, color_list, pt_size=2, iflabel=True, xoff=0, yoff=-0.1
):
    """Plot top n popular movie with Vs"""
    labels = hv.Labels(
        {("proj_1", "proj_2"): df, "text": df.TITLE}, ["proj_1", "proj_2"], "text",
    )

    if iflabel:
        return (
            hv.Points(df, kdims=["proj_1", "proj_2"], label=legend_label) * labels
        ).opts(
            opts.Points(color=color_list, fill_alpha=0.5, line_width=2, size=pt_size),
            opts.Labels(text_font_size="10pt", xoffset=xoff, yoffset=yoff),
        )
    else:
        return (hv.Points(df, kdims=["proj_1", "proj_2"], label=legend_label)).opts(
            color=color_list, fill_alpha=0.5, line_width=2
        )
    
def get_legend_highlow(rankcols, mostorleast="most"):
    """Determine how to phrase legend"""

    if mostorleast == "most":
        morl = "most"
        horl = "hights"
    else:
        morl = "least"
        horl = "lowest"

    if rankcols == ["RATING_COUNTS", "AVG_RATING"]:
        leg = f"{morl} & {horl} rated"
    elif rankcols == ["AVG_RATING", "RATING_COUNTS"]:
        leg = f"{horl} & {morl} rated"
    elif rankcols == ["AVG_RATING"] or rankcols == "AVG_RATING":
        leg = f"{horl} rated"
    elif rankcols == ["RATING_COUNTS"] or rankcols == "RATING_COUNTS":
        leg = f"{morl} rated"

    return leg


def get_dfs(
    comb_V_df, rankcols, three_genres, topn=10, cutoff=5, mostorleast="most",
):
    """Get the list of dfs for given genre, rating, popularities"""
    g_dfs = [None] * 3
    for g, genre in enumerate(three_genres):
        g_dfs[g] = get_topn_pop_n_rate_proj(
            comb_V_df,
            rankcols,
            topn=topn,
            cutoff=cutoff,
            mostorleast=mostorleast,
            genres=genre,
        )
    return g_dfs


def get_g_handpick_dfs(df, chosen_id_dict):
    """Return sliced dataframe with chosen IDs"""
    dfs = [None] * len(chosen_id_dict)

    for i, (k, v) in enumerate(chosen_id_dict.items()):
        dfs[i] = df[df.MOVIE_ID.isin(v)]

    return list(chosen_id_dict.keys()), dfs


def plot_overlay_pt(
    comb_V_df,
    comb_U_df,
    three_genres,
    rankcols,
    ifhandpick=False,
    chosen_id_dict=None,
    topn=10,
    cutoff=5,
    mostorleast="most",
    iflabel=False,
    xoff=0,
    yoff=-0.1,
):

    """generate point plots overlays with 3 chosen genres"""

    add_colors = sns.color_palette("YlOrBr", 10).as_hex()

    topn_pop_V_proj = get_topn_pop_n_rate_proj(comb_V_df, "RATING_COUNTS")
    # df, rankcols, topn=10, cutoff=5, mostorleast="most", genres=None
    topn_rate_V_proj_5_cut = get_topn_pop_n_rate_proj(comb_V_df, "AVG_RATING")

    """g_dfs = [None] * 3
    for g, genre in enumerate(three_genres):
        g_dfs[g] = get_topn_pop_n_rate_proj(
            comb_V_df,
            rankcols,
            topn=topn,
            cutoff=cutoff,
            mostorleast=mostorleast,
            genres=genre,
        )"""

    if ifhandpick and chosen_id_dict != None:
        three_genres, g_dfs = get_g_handpick_dfs(comb_V_df, chosen_id_dict)
        leg = "Chosen"
        # iflabel = True
    else: 
        g_dfs = get_dfs(
            comb_V_df,
            rankcols,
            three_genres,
            topn=topn,
            cutoff=cutoff,
            mostorleast=mostorleast)
        leg = get_legend_highlow(rankcols, mostorleast=mostorleast)
    
    overlay_pt = hv.render(
        plot_V_pts(comb_V_df)
        * plot_U_pts(comb_U_df)
        * plot_selected_V(
            topn_pop_V_proj, 
            "most rated", 
            list(add_colors)[1], 
            pt_size=10, 
            iflabel=True, 
            xoff=xoff,
            yoff=yoff,
        )
        * plot_selected_V(
            topn_rate_V_proj_5_cut,
            "highest rating (≥5 ratings)",
            list(add_colors)[3],
            pt_size=8,
            iflabel=True,
            xoff=xoff,
            yoff=yoff,
        )
        * plot_selected_V(
            g_dfs[0],
            f"{leg} {three_genres[0]}",
            list(add_colors)[5],
            pt_size=6,
            iflabel=iflabel,
            xoff=xoff,
            yoff=yoff,
        )
        * plot_selected_V(
            g_dfs[1],
            f"{leg} {three_genres[1]}",
            list(add_colors)[7],
            pt_size=4,
            iflabel=iflabel,
            xoff=xoff,
            yoff=yoff,
        )
        * plot_selected_V(
            g_dfs[2],
            f"{leg} {three_genres[2]}",
            list(add_colors)[9],
            pt_size=2,
            iflabel=iflabel,
            xoff=xoff,
            yoff=yoff,
        )
    )

    return overlay_pt


def plot_layout(counts_ecdf, count_hist, overlay_pt, avg_hist_p, rate_ecdf):
    """Plot the layout with points, ecdfs, and histograms"""
    layout = gridplot(
        [
            [counts_ecdf, None, None],
            [count_hist, None, None],
            [overlay_pt, avg_hist_p, rate_ecdf],
        ],
        merge_tools=True,
    )
    bokeh.io.show(layout)
    
    return layout

In [ ]:
def get_UV2layout(
    U,
    V,
    data,
    movies,
    three_genres,
    rankcols,
    title_details,
    ifhandpick=False,
    chosen_id_dict=None,
    topn=10,
    cutoff=5,
    mostorleast="most",
    iflabel=False,
    xoff=0,
    yoff=-0.1,
    needproj=True,
):
    """Put together the whole process"""
    if needproj:
        U_proj, V_proj = get_UV_proj(U, V)
    else:
        U_proj, V_proj = (U, V)

    comb_V_df = comb_proj_rate_pop(V_proj, data, movies)
    comb_U_df = comb_proj_rate_pop(U_proj, data, movies)

    counts_ecdf, rate_ecdf = plot_ecdfs(data, movies, ifshow=False)

    meta_rate_pop = comb_rate_pop(data, movies)
    count_hist, counts_ecdf, avg_hist_p, rate_ecdf = get_hists_ecdfs(
        meta_rate_pop, title_details
    )

    overlay_pt = plot_overlay_pt(
        comb_V_df,
        comb_U_df,
        three_genres,
        rankcols,
        ifhandpick=ifhandpick,
        chosen_id_dict=chosen_id_dict,
        topn=topn,
        cutoff=cutoff,
        mostorleast=mostorleast,
        iflabel=iflabel,
        xoff=xoff,
        yoff=yoff,
    )
    
    layout = plot_layout(counts_ecdf, count_hist, overlay_pt, avg_hist_p, rate_ecdf)
    return (
        U_proj,
        V_proj,
        comb_V_df,
        comb_U_df,
        counts_ecdf,
        count_hist,
        overlay_pt,
        avg_hist_p,
        rate_ecdf,
        layout,
    )

In [ ]:
def get_list_movies(data, head_no):
    """ selects the first head_no items from data and returns 3 lists of ids, genre and titles"""   

    list_movies_id = [d[1] for d in data.head(head_no).values]

    list_movies_genre = []
    list_movies_titles = []
    for idx in list_movies_id:
        movie_id_df = movies[movies["MOVIE_ID"] == idx]
        list_cols = []
        for i, v in enumerate(movie_id_df.values[0,:]):
            if v == 1:
                list_cols.append(movie_id_df.columns[i])
        list_movies_genre.append(list_cols)
        list_movies_titles.append(movie_id_df.values[0,1])
        
    return list_movies_id, list_movies_genre, list_movies_titles

# Some visualizations
def visualize(V_proj, movie_titles, movie_genre):
    """ Plots V_proj with annotations from the lists of ids, genre and titles"""   

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.axhline(y=0, color="r")
    ax.axvline(x=0, color="r")
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    m_proj0 = np.mean(V_proj[0,:])
    m_proj1 = np.mean(V_proj[1,:])
    ax.plot(V_proj[0,:] - m_proj0, V_proj[1,:] - m_proj1,"*")
    ax.set_xlabel("V proj 0")
    ax.set_ylabel("V proj 1")
    offset = 0.01
    for i, txt in enumerate(list_movies_titles):
        ax.annotate(txt, (V_proj[0][i] - m_proj0 + offset, V_proj[1][i] - m_proj1 + offset), fontsize=8)

    for i, txt in enumerate(list_movies_genre):
        ax.annotate(txt, (V_proj[0][i] - m_proj0, V_proj[1][i] - m_proj1 - 5*offset), fontsize=8)

    plt.show()


In [ ]:
Y_train, Y_test, M, N = get_MNY(Y_train_df, Y_test_df)

### Choose movies and matrix

In [ ]:
chosen_id_dict = {
    "COMEDY": [1, 70, 94, 168, 202, 257, 894, 902, 820, 173],
    "ROMANCE": [50, 69, 70, 133, 155, 161, 172, 213, 313, 465],
    "SCI-FI": [89, 96, 109, 176, 195, 228, 229, 343, 429, 449],
}
three_genres = ["COMEDY", "ROMANCE", "SCI-FI"]
rankcols = ["AVG_RATING"]

## Method One (No bias)

### HW (No bias)

In [ ]:
def run_non_bias_model(Y_train, M, N, Ks, eta):
    """The non-biased HW"""
    # print("Factorizing with ", M, " users, ", N, " movies.")
    reg = 0

    # Use to compute Ein and Eout
    U1, V1, err_training = train_model(M, N, Ks, eta, reg, Y_train)
    err_test = get_err(U1, V1, Y_test)

    print("Err training", err_training)
    print("Err testing", err_test)
    # print("dim U [m x k]", U1.shape)
    # print("dim V [k x n]", V1.shape)

    return U1, V1, err_training, err_test

#### Optimize eta for non-biase HW modification

In [ ]:
def opt_non_bias_model(eta: float) -> float:

    U1, V1, err_training, err_test = run_non_bias_model(
        Y_train, M, N, Ks, eta
    )

    return err_test

# Instrumentation class is used for functions with multiple inputs
# (positional and/or keywords)

Ks = 20

# a greedy parametrization, we can reduce
parametrization = ng.p.Instrumentation(
    eta=ng.p.Scalar(lower=0.01, upper=0.1),  # given 0.03
)

optimizer = ng.optimizers.NGOpt(parametrization=parametrization, budget=10)
recommendation = optimizer.minimize(opt_non_bias_model, verbosity=2)

#### Get UV, error, and plots with opt eta and reg for non-biased HW modification

In [ ]:
 Ks = 20
 eta = 0.04851345326663499
 U1, V1, err_training, err_test = run_non_bias_model(Y_train, M, N, Ks, eta)

In [ ]:
(
    U1_proj,
    V1_proj,
    comb_V1_df,
    comb_U1_df,
    counts_ecdf,
    count_hist,
    overlay_pt,
    avg_hist_p,
    rate_ecdf,
    layout,
) = get_UV2layout(
    U1,
    V1,
    data,
    movies,
    three_genres,
    rankcols,
    "without bias",
    ifhandpick=True,
    chosen_id_dict=chosen_id_dict,
    topn=10,
    cutoff=5,
    mostorleast=None,
    xoff=0.2,
    yoff=-0.1
)

### Surprise (No bias)

#### Optimize eta for non-biased with surprise

In [ ]:
def optimize_non_bias_surprise(
    n_factors: int, n_epochs: int, lr_all: float, 
) -> float:

    """n_factors – The number of factors. Default is 100.
    n_epochs – The number of iteration of the SGD procedure. Default is 20.
    biased (bool) – Whether to use baselines (or biases). See note above. Default is True.
    init_mean – The mean of the normal distribution for factor vectors initialization. Default is 0.
    init_std_dev – The standard deviation of the normal distribution for factor vectors initialization. Default is 0.1.
    lr_all – The learning rate for all parameters. Default is 0.005.
    reg_all – The regularization term for all parameters. Default is 0.02.
    lr_bu – The learning rate for 𝑏𝑢. Takes precedence over lr_all if set. Default is None.
    lr_bi – The learning rate for 𝑏𝑖. Takes precedence over lr_all if set. Default is None.
    lr_pu – The learning rate for 𝑝𝑢. Takes precedence over lr_all if set. Default is None.
    lr_qi – The learning rate for 𝑞𝑖. Takes precedence over lr_all if set. Default is None.
    reg_bu – The regularization term for 𝑏𝑢. Takes precedence over reg_all if set. Default is None.
    reg_bi – The regularization term for 𝑏𝑖. Takes precedence over reg_all if set. Default is None.
    reg_pu – The regularization term for 𝑝𝑢. Takes precedence over reg_all if set. Default is None.
    reg_qi – The regularization term for 𝑞𝑖. Takes precedence over reg_all if set. Default is None.
    random_state (int, RandomState instance from numpy, or None) – 
    Determines the RNG that will be used for initialization. If int, random_state will be used as a seed for a new RNG. 
    This is useful to get the same initialization over multiple calls to fit(). 
    If RandomState instance, this same instance is used as RNG. 
    If None, the current RNG from numpy is used. Default is None.
    verbose – If True, prints the current epoch. Default is False."""

    surprise_rmse, U, V = use_surprise(
        Y_train_df,
        n_factors,
        n_epochs,
        lr_all,
        0,
        ifbiased=False,
        ifreg_bubi=False,
    )
    # Y_train_df, n_factors, n_epochs, lr_all, reg_all, ifbiased, ifreg_bubi
    return surprise_rmse

# Instrumentation class is used for functions with multiple inputs
# (positional and/or keywords)

# a greedy parametrization, we can reduce
parametrization = ng.p.Instrumentation(
    n_factors=ng.p.Scalar(lower=50, upper=200).set_integer_casting(),
    n_epochs=ng.p.Scalar(lower=10, upper=50).set_integer_casting(),
    lr_all=ng.p.Scalar(lower=0.001, upper=0.1),
)

optimizer = ng.optimizers.NGOpt(parametrization=parametrization, budget=10)
recommendation = optimizer.minimize(optimize_non_bias_surprise, verbosity=2)

#### Get UV, error, and plots with opt eta for non-biased with surprise

In [ ]:
n_factors = 149
n_epochs = 31
lr_all = 0.05832950360579468
reg_all = 0

surprise_rmse_notreg_bias, U4_proj, V4_proj  = use_surprise(
        Y_train_df,
        n_factors,
        n_epochs,
        lr_all,
        reg_all,
        ifbiased=False,
        ifreg_bubi=False,
    )

In [ ]:
(
    U4_proj,
    V4_proj,
    comb_V4_df,
    comb_U4_df,
    counts_ecdf,
    count_hist,
    overlay_pt,
    avg_hist_p,
    rate_ecdf,
    layout,
) = get_UV2layout(
    U4_proj,
    V4_proj,
    data,
    movies,
    three_genres,
    rankcols,
    "Suprise without bias",
    ifhandpick=True,
    chosen_id_dict=chosen_id_dict,
    topn=10,
    cutoff=5,
    mostorleast=None,
    xoff=0,
    yoff=-0.025,
    needproj=False
)

## Method Two (Not regularized bias)

### HW modification (Not regularized bias)

In [ ]:
def add_bias_UV(U, V):
    """Add biases to U and V"""
    return (
        np.hstack([np.ones((U.shape[0], 1)), U]),
        np.vstack([np.ones((1, V.shape[1])), V]),
    )

In [ ]:
def train_nonreg_bias_model(M, N, K, eta, reg, Y, eps=0.0001, max_epochs=300):
    """
    Modify the train_model function
    """
    
    np.random.seed(42)
    
    # Initialize U, V  
    U = np.random.random((M,K)) - 0.5
    V = np.random.random((K,N)) - 0.5
    
    # Add bias
    U, V = add_bias_UV(U, V)
    
    size = Y.shape[0]
    delta = None
    indices = np.arange(size)    
    for epoch in range(max_epochs):
        # Run an epoch of SGD
        before_E_in = get_err(U, V, Y, reg)
        np.random.shuffle(indices)
        for ind in indices:
            (i,j, Yij) = Y[ind]
            # Update U[i], V[j]
            U[i-1] = grad_U(U[i-1], Yij, V[:,j-1], reg, eta)
            V[:,j-1] = grad_V(V[:,j-1], Yij, U[i-1], reg, eta);
        # At end of epoch, print E_in
        E_in = get_err(U, V, Y, reg)
        print("Epoch %s, E_in (regularized MSE): %s"%(epoch + 1, E_in))

        # Compute change in E_in for first epoch
        if epoch == 0:
            delta = before_E_in - E_in

        # If E_in doesn"t decrease by some fraction <eps>
        # of the initial decrease in E_in, stop early            
        elif before_E_in - E_in < eps * delta:
            break
    return U, V, get_err(U, V, Y)

In [ ]:
def run_nonreg_bias_model(M, N, Ks, eta, reg, Y_train, Y_test):
    """Run training with non-reg biase"""
    # print("Factorizing with ", M, " users, ", N, " movies.")
    
    U2, V2, err_training = train_nonreg_bias_model(M, N, Ks, eta, reg, Y_train)
    err_test = get_err(U2, V2, Y_test)
    
    print("Err training", err_training)
    print("Err testing", err_test)
    # print("dim U [m x k]", U2.shape)
    # print("dim V [k x n]", V2.shape)
    
    return U2, V2, err_training, err_test

#### Optimize eta and reg for not regularized bias HW modification

In [ ]:
def opt_nonreg_bias_model(eta: float, reg: float) -> float:

    U2, V2, err_training, err_test = run_nonreg_bias_model(
        M, N, Ks, eta, reg, Y_train, Y_test
    )

    return err_test


# Instrumentation class is used for functions with multiple inputs
# (positional and/or keywords)

Ks = 20

# a greedy parametrization, we can reduce
parametrization = ng.p.Instrumentation(
    eta=ng.p.Scalar(lower=0.01, upper=0.1),  # given 0.03
    reg=ng.p.Scalar(lower=0.01, upper=0.1),
)

optimizer = ng.optimizers.NGOpt(parametrization=parametrization, budget=10)
recommendation = optimizer.minimize(opt_nonreg_bias_model, verbosity=2)

#### Get UV, error, and plots with opt eta and reg for not regularized bias HW modification

In [ ]:
Ks = 20
eta = 0.013227454444331396
reg = 0.06917747048954087
U2, V2, err_training, err_test = run_nonreg_bias_model(
    M, N, Ks, eta, reg, Y_train, Y_test
)

In [ ]:
(
    U2_proj,
    V2_proj,
    comb_V2_df,
    comb_U2_df,
    counts_ecdf,
    count_hist,
    overlay_pt,
    avg_hist_p,
    rate_ecdf,
    layout,
) = get_UV2layout(
    U2,
    V2,
    data,
    movies,
    three_genres,
    rankcols,
    "with non-regularized bias",
    ifhandpick=True,
    chosen_id_dict=chosen_id_dict,
    topn=10,
    cutoff=5,
    mostorleast=None,
    xoff=0.15,
    yoff=-0.06
)

### Surprise (Not regularized bias)

#### Optimize eta and reg for not regularized bias with surprise

In [ ]:
def optimize_nonreg_bias_surprise(
    n_factors: int, n_epochs: int, lr_all: float, reg_all: int
) -> float:

    """n_factors – The number of factors. Default is 100.
    n_epochs – The number of iteration of the SGD procedure. Default is 20.
    biased (bool) – Whether to use baselines (or biases). See note above. Default is True.
    init_mean – The mean of the normal distribution for factor vectors initialization. Default is 0.
    init_std_dev – The standard deviation of the normal distribution for factor vectors initialization. Default is 0.1.
    lr_all – The learning rate for all parameters. Default is 0.005.
    reg_all – The regularization term for all parameters. Default is 0.02.
    lr_bu – The learning rate for 𝑏𝑢. Takes precedence over lr_all if set. Default is None.
    lr_bi – The learning rate for 𝑏𝑖. Takes precedence over lr_all if set. Default is None.
    lr_pu – The learning rate for 𝑝𝑢. Takes precedence over lr_all if set. Default is None.
    lr_qi – The learning rate for 𝑞𝑖. Takes precedence over lr_all if set. Default is None.
    reg_bu – The regularization term for 𝑏𝑢. Takes precedence over reg_all if set. Default is None.
    reg_bi – The regularization term for 𝑏𝑖. Takes precedence over reg_all if set. Default is None.
    reg_pu – The regularization term for 𝑝𝑢. Takes precedence over reg_all if set. Default is None.
    reg_qi – The regularization term for 𝑞𝑖. Takes precedence over reg_all if set. Default is None.
    random_state (int, RandomState instance from numpy, or None) – 
    Determines the RNG that will be used for initialization. If int, random_state will be used as a seed for a new RNG. 
    This is useful to get the same initialization over multiple calls to fit(). 
    If RandomState instance, this same instance is used as RNG. 
    If None, the current RNG from numpy is used. Default is None.
    verbose – If True, prints the current epoch. Default is False."""

    surprise_rmse, U, V = use_surprise(
        Y_train_df,
        n_factors,
        n_epochs,
        lr_all,
        reg_all,
        ifbiased=True,
        ifreg_bubi=False,
    )
    # Y_train_df, n_factors, n_epochs, lr_all, reg_all, ifbiased, ifreg_bubi
    return surprise_rmse

# Instrumentation class is used for functions with multiple inputs
# (positional and/or keywords)

# a greedy parametrization, we can reduce
parametrization = ng.p.Instrumentation(
    n_factors=ng.p.Scalar(lower=50, upper=200).set_integer_casting(),
    n_epochs=ng.p.Scalar(lower=10, upper=50).set_integer_casting(),
    lr_all=ng.p.Scalar(lower=0.001, upper=0.1),
    reg_all=ng.p.Scalar(lower=0.01, upper=0.1),
)

optimizer = ng.optimizers.NGOpt(parametrization=parametrization, budget=10)
recommendation = optimizer.minimize(optimize_nonreg_bias_surprise, verbosity=2)

#### Get UV, error, and plots with opt eta and reg for not regularized bias with surprise

In [ ]:
n_factors = 125
n_epochs = 35
lr_all = 0.009813324125778348
reg_all = 0.09073507141179653

surprise_rmse_notreg_bias, U5_proj, V5_proj  = use_surprise(
        Y_train_df,
        n_factors,
        n_epochs,
        lr_all,
        reg_all,
        ifbiased=True,
        ifreg_bubi=False,
    )

In [ ]:
(
    U5_proj,
    V5_proj,
    comb_V5_df,
    comb_U5_df,
    counts_ecdf,
    count_hist,
    overlay_pt,
    avg_hist_p,
    rate_ecdf,
    layout,
) = get_UV2layout(
    U5_proj,
    V5_proj,
    data,
    movies,
    three_genres,
    rankcols,
    "Suprise with non-regulated bias",
    ifhandpick=True,
    chosen_id_dict=chosen_id_dict,
    topn=10,
    cutoff=5,
    mostorleast=None,
    xoff=0,
    yoff=-0.025,
    needproj=False
)

## Method Three (Regularized bias)

In [ ]:
def run_reg_bias_model(M, N, Ks, eta, reg, Y_train, Y_test):
    """Run training with non-reg biase"""
    # print("Factorizing with ", M, " users, ", N, " movies.")

    U3, V3, a, b, err_training = train_model_biased(
        M, N, K, eta, reg, Y_train, eps=0.0001, max_epochs=300
    )
    err_test = get_err_biased(U3, V3, Y_test, a, b, reg)

    print("Err training", err_training)
    print("Err testing", err_test)

    return U3, V3, err_training, err_test

### Surprise (Regularized bias)

#### Optimize eta and reg for regularized bias with surprise

In [ ]:
def optimize_reg_bias_surprise(
    n_factors: int, n_epochs: int, lr_all: float, reg_all: int
) -> float:

    """n_factors – The number of factors. Default is 100.
    n_epochs – The number of iteration of the SGD procedure. Default is 20.
    biased (bool) – Whether to use baselines (or biases). See note above. Default is True.
    init_mean – The mean of the normal distribution for factor vectors initialization. Default is 0.
    init_std_dev – The standard deviation of the normal distribution for factor vectors initialization. Default is 0.1.
    lr_all – The learning rate for all parameters. Default is 0.005.
    reg_all – The regularization term for all parameters. Default is 0.02.
    lr_bu – The learning rate for 𝑏𝑢. Takes precedence over lr_all if set. Default is None.
    lr_bi – The learning rate for 𝑏𝑖. Takes precedence over lr_all if set. Default is None.
    lr_pu – The learning rate for 𝑝𝑢. Takes precedence over lr_all if set. Default is None.
    lr_qi – The learning rate for 𝑞𝑖. Takes precedence over lr_all if set. Default is None.
    reg_bu – The regularization term for 𝑏𝑢. Takes precedence over reg_all if set. Default is None.
    reg_bi – The regularization term for 𝑏𝑖. Takes precedence over reg_all if set. Default is None.
    reg_pu – The regularization term for 𝑝𝑢. Takes precedence over reg_all if set. Default is None.
    reg_qi – The regularization term for 𝑞𝑖. Takes precedence over reg_all if set. Default is None.
    random_state (int, RandomState instance from numpy, or None) – 
    Determines the RNG that will be used for initialization. If int, random_state will be used as a seed for a new RNG. 
    This is useful to get the same initialization over multiple calls to fit(). 
    If RandomState instance, this same instance is used as RNG. 
    If None, the current RNG from numpy is used. Default is None.
    verbose – If True, prints the current epoch. Default is False."""

    surprise_rmse, U, V = use_surprise(
        Y_train_df,
        n_factors,
        n_epochs,
        lr_all,
        reg_all,
        ifbiased=True,
        ifreg_bubi=True,
    )
    # Y_train_df, n_factors, n_epochs, lr_all, reg_all, ifbiased, ifreg_bubi
    return surprise_rmse

# Instrumentation class is used for functions with multiple inputs
# (positional and/or keywords)

# a greedy parametrization, we can reduce
parametrization = ng.p.Instrumentation(
    n_factors=ng.p.Scalar(lower=50, upper=200).set_integer_casting(),
    n_epochs=ng.p.Scalar(lower=10, upper=50).set_integer_casting(),
    lr_all=ng.p.Scalar(lower=0.001, upper=0.1),
    reg_all=ng.p.Scalar(lower=0.01, upper=0.1),
)

optimizer = ng.optimizers.NGOpt(parametrization=parametrization, budget=10)
recommendation = optimizer.minimize(optimize_reg_bias_surprise, verbosity=2)

#### Get UV, error, and plots with opt eta and reg for regularized bias with surprise

In [ ]:
n_factors = 110
n_epochs = 21
lr_all = 0.029661498219795867
reg_all = 0.050718435030741

In [ ]:
surprise_rmse_notreg_bias, U6_proj, V6_proj  = use_surprise(
        Y_train_df,
        n_factors,
        n_epochs,
        lr_all,
        reg_all,
        ifbiased=True,
        ifreg_bubi=True,
    )

In [ ]:
(
    U6_proj,
    V6_proj,
    comb_V6_df,
    comb_U6_df,
    counts_ecdf,
    count_hist,
    overlay_pt,
    avg_hist_p,
    rate_ecdf,
    layout,
) = get_UV2layout(
    U6_proj,
    V6_proj,
    data,
    movies,
    three_genres,
    rankcols,
    "Suprise with non-regulated bias",
    ifhandpick=True,
    chosen_id_dict=chosen_id_dict,
    topn=10,
    cutoff=5,
    mostorleast=None,
    xoff=0,
    yoff=-0.025,
    needproj=False
)